In [1]:
from typing import List
from langchain_core.callbacks import CallbackManagerForRetrieverRun
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from ragbuilder.generation.utils import dummyRetriever

In [2]:
strings = [
    "A young woman.",
    """## Chunk 3: The Locket’s Secret

Clara awoke to a sound—footsteps. Her eyes shot open. In the dim light, she saw a man standing at the door, silhouetted against the night. His green eyes gleamed in the shadows.

“Who are you?” she stammered, clutching her locket.

“I could ask you the same,” the man replied, stepping forward. “This is my family’s cabin. What are you doing here?”
"""
]
documents = dummyRetriever.strings_to_documents(strings)
retriever = dummyRetriever(documents=documents, k=3)

In [3]:
retriever.invoke("Clara")

[Document(page_content='## Chunk 3: The Locket’s Secret\n\nClara awoke to a sound—footsteps. Her eyes shot open. In the dim light, she saw a man standing at the door, silhouetted against the night. His green eyes gleamed in the shadows.\n\n“Who are you?” she stammered, clutching her locket.\n\n“I could ask you the same,” the man replied, stepping forward. “This is my family’s cabin. What are you doing here?”\n')]